## Operators in MongoDB

Disclaimer: the following table doesn't imply any horizontal relationship between "first level" and "second level" operators. 

|"First level"|"Second level"|
|---|---|
|match|elemMatch, type, size|
|group|sum, avg, max, min|
|unwind|push|
|sort|first, last|
|project|gte, gt, lte, lt|
|limit|and, or, cond|
|replaceRoot|mergeObjects|

Let's re-define a function to perform queries by PyMongo using ``aggregate`` method.

In [1]:
import pandas as pd
import pymongo
import seaborn as sns

def query(database, collection, pipeline, host='localhost', port=27017):
    try:
        client = pymongo.MongoClient(host=host, port=port)
        db = client[database]
        cll = db[collection]
    except (Exception) as e:
        print(e)
    
    else:
        try:
            cursor = cll.aggregate(pipeline)
            return pd.DataFrame(list(cursor))
        except (Exception) as e:
            print(e)
        finally:
            client.close()            

NOTE: opening and closing a connection with ``MongoClient`` everytime we need to perform a query is not efficient.
> POSSIBLE SOLUTION: define two different functions.

In [2]:
def mongo_connect(database, collection, host = "localhost", port = 27017):
    try:
        client = pymongo.MongoClient(host=host, port=port)
        db = client[database]
        cll = db[collection]
        return cll, client
    except (Exception) as e:
        print(e)

def query(cll, pipeline):
    
    try:
        cursor = cll.aggregate(pipeline, allowDiskUse = True)
        return pd.DataFrame(list(cursor))
    except (Exception) as e:
        print(e)

In [3]:
# My parameters
database = 'instacart'
collection = 'orders'

In [4]:
cll, client = mongo_connect(database, collection)
pipeline =[
    {"$match": {"order_dow":2, "order_hour_of_day":{"$gte":20}}},
    {"$project": {"_id": 0, "products":0}}
]

In [5]:
query(cll, pipeline)

,order_id,user_id,order_number,order_dow,order_hour_of_day,days_since_prior_order
0,2111787,41,2,2,20,24.0
1,1899315,47,5,2,22,14.0
2,1102893,54,51,2,23,2.0
3,1325316,54,78,2,23,1.0
4,1456703,54,73,2,23,1.0
...,...,...,...,...,...,...
11419,237799,60023,13,2,20,4.0
11420,1891538,60043,24,2,20,9.0
11421,1864445,60049,31,2,20,11.0
11422,2779687,60052,54,2,21,2.0


In [6]:
client.close()

### Exercise 2.1
##### Retrieve the 10 aisles with most purchases at nighttime (from 9 pm to 6 am) and compare them with the aisles having the highest number of purchases at daytime

In [7]:
cll, client = mongo_connect(database, collection)

In [ ]:
# unwind
# match lt 6 or gt 21
# group products.aisle sum: 1
# sort count -1
# limit 10






In [8]:
night_pipeline = [
    {"$match": {"$or":[{"order_hour_of_day":{"$gte":21}}, {"order_hour_of_day":{"$lte":6}}]}},
    {"$unwind": "$products"},
    {"$group": {"_id": "$products.aisle", "count": {"$sum": 1}}},
    {"$sort": {"count": -1}},
    {"$limit": 10}
]

In [9]:
day_pipeline = [
    {"$match": {"order_hour_of_day":{"$lt":21, "$gt":6}}},
    {"$unwind": "$products"},
    {"$group": {"_id": "$products.aisle", "count": {"$sum": 1}}},
    {"$sort": {"count": -1}},
    {"$limit": 10}
]

In [10]:
night_aisles = query(cll, night_pipeline)
day_aisles = query(cll, day_pipeline)

In [11]:
pd.merge(night_aisles, day_aisles, left_index = True, right_index = True, how = "inner")

,_id_x,count_x,_id_y,count_y
0,fresh fruits,89658,fresh fruits,1009329
1,fresh vegetables,87451,fresh vegetables,950109
2,packaged vegetables fruits,43762,packaged vegetables fruits,490226
3,yogurt,35784,yogurt,398490
4,packaged cheese,22441,packaged cheese,273474
5,milk,20550,milk,246162
6,water seltzer sparkling water,16935,water seltzer sparkling water,237875
7,soy lactosefree,15258,chips pretzels,203181
8,chips pretzels,14088,soy lactosefree,179173
9,frozen produce,13861,bread,162638


### Digression: Replace Root

https://www.mongodb.com/docs/manual/reference/operator/aggregation/replaceRoot/

In [12]:
match = {"$match": {"order_id": 473748}}
unwind = {"$unwind": "$products"}

# replace root
replaceRoot = {
    "$replaceRoot": {
        "newRoot": {
            "$mergeObjects": [
                {
                    "_id": "$_id",
                    'order_id': "$order_id"
                },
                "$products"
            ]
        }
    }
}

df_with_replace = query(cll, [match, unwind, replaceRoot])
df_with_replace.head()

,_id,order_id,product_id,product_name,aisle_id,aisle,department_id,department,add_to_cart_order,reordered
0,60af5b022fbbe97183553d36,473748,6188,Organic Apple & Butternut Squash Baby Food,92,baby food formula,18,babies,10,NaN
1,60af5b022fbbe97183553d36,473748,6347,Unsweetened Almond Milk,91,soy lactosefree,16,dairy eggs,14,NaN
2,60af5b022fbbe97183553d36,473748,7051,"Happy Tot Banana, Peach, Prune & Coconut Organ...",92,baby food formula,18,babies,2,NaN
3,60af5b022fbbe97183553d36,473748,11712,Cage Free Large White Eggs,86,eggs,16,dairy eggs,13,1.0
4,60af5b022fbbe97183553d36,473748,13176,Bag of Organic Bananas,24,fresh fruits,4,produce,1,1.0


In [13]:
replaceRoot = {
    "$replaceRoot": {
        "newRoot": {
            "_id":"$_id",
            "products":"$products.product_name"
        }
    }
}


df_with_replace2 = query(cll, [match, unwind, replaceRoot])
df_with_replace2.head()

,_id,products
0,60af5b022fbbe97183553d36,Organic Apple & Butternut Squash Baby Food
1,60af5b022fbbe97183553d36,Unsweetened Almond Milk
2,60af5b022fbbe97183553d36,"Happy Tot Banana, Peach, Prune & Coconut Organ..."
3,60af5b022fbbe97183553d36,Cage Free Large White Eggs
4,60af5b022fbbe97183553d36,Bag of Organic Bananas


In [14]:
# Unwinding the array field only divides the elements of the array into different records,
# but it does not make the attributes of such elements directly accessible
df_no_replace = query(cll, [match, unwind])
df_no_replace.head()

,_id,order_id,user_id,order_number,order_dow,order_hour_of_day,days_since_prior_order,products
0,60af5b022fbbe97183553d36,473748,40904,19,4,10,14.0,"{'product_id': 6188, 'product_name': 'Organic ..."
1,60af5b022fbbe97183553d36,473748,40904,19,4,10,14.0,"{'product_id': 6347, 'product_name': 'Unsweete..."
2,60af5b022fbbe97183553d36,473748,40904,19,4,10,14.0,"{'product_id': 7051, 'product_name': 'Happy To..."
3,60af5b022fbbe97183553d36,473748,40904,19,4,10,14.0,"{'product_id': 11712, 'product_name': 'Cage Fr..."
4,60af5b022fbbe97183553d36,473748,40904,19,4,10,14.0,"{'product_id': 13176, 'product_name': 'Bag of ..."


In [15]:
cursor = cll.find()

In [16]:
cursor.next()

{'_id': ObjectId('60af57782fbbe971834ae2a6'),
 'order_id': 2539329,
 'user_id': 1,
 'order_number': 1,
 'order_dow': 2,
 'order_hour_of_day': 8,
 'products': [{'product_id': 196,
   'product_name': 'Soda',
   'aisle_id': 77,
   'aisle': 'soft drinks',
   'department_id': 7,
   'department': 'beverages',
   'add_to_cart_order': 1},
  {'product_id': 12427,
   'product_name': 'Original Beef Jerky',
   'aisle_id': 23,
   'aisle': 'popcorn jerky',
   'department_id': 19,
   'department': 'snacks',
   'add_to_cart_order': 3},
  {'product_id': 14084,
   'product_name': 'Organic Unsweetened Vanilla Almond Milk',
   'aisle_id': 91,
   'aisle': 'soy lactosefree',
   'department_id': 16,
   'department': 'dairy eggs',
   'add_to_cart_order': 2},
  {'product_id': 26088,
   'product_name': 'Aged White Cheddar Popcorn',
   'aisle_id': 23,
   'aisle': 'popcorn jerky',
   'department_id': 19,
   'department': 'snacks',
   'add_to_cart_order': 4},
  {'product_id': 26405,
   'product_name': 'XL Pick-A-S

### Exercise 2.2
##### Count the number of orders that contain at least 20 products

In [ ]:
# unwind
# group order_id sum:1
# match count gte 20





In [17]:
match = {"$match": {"products": {"$type": "array"}}}
project = {"$project": {"order_id": 1, "n_products":{"$size":"$products"}}}
match1 = {"$match": {"n_products": {"$gte": 20}}}
group = {"$group":{"_id": None, "count": {"$sum": 1}}}

query(cll, [match, project, match1, group])

,_id,count
0,None,106648


In [18]:
match = {"$match": {"products": {"$type": "array"}}}
project = {"$project": {"order_id": 1, "n_products":{"$max":"$products.add_to_cart_order"}}}
match1 = {"$match": {"n_products": {"$gte": 20}}}
group = {"$group":{"_id": None, "count": {"$sum": 1}}}

query(cll, [match, project, match1, group])

,_id,count
0,None,106648


In [19]:
unwind = {"$unwind" : "$products"}
group = {"$group" : {"_id" : "$_id", "Number of products" : { "$sum": 1 }}}
match = {"$match" : {"Number of products" : {"$gte" : 20}}}
group1 = {"$group" : {"_id" : None, "Total orders with > 20 products" : { "$sum": 1 }}}

query(cll, [unwind, group, match, group1])

,_id,Total orders with > 20 products
0,None,106648


In [20]:
query(cll, [{"$group": {"_id": None, "Total orders": {"$sum": 1}}}])

,_id,Total orders
0,None,1000000


NOTE: ``$sum`` returns the sum of the numeric values that result from applying the following expression to each document in a group of documents. In our example the expression is simply equal to 1 for each document, so ``$sum`` will aggregate a value of 1 for each document in the group, thus yielding the total number of documents per group.

SECOND NOTE: ``$size`` operator wouldn't work, since it doesn't accept inequalities as argument, as stated in the documentation (https://www.mongodb.com/docs/manual/reference/operator/query/size/)

In [22]:
query(cll, [{"$match":{"products":{"$size":{"$gte":20}}}}])

$size needs a number, full error: {'ok': 0.0, 'errmsg': '$size needs a number', 'code': 2, 'codeName': 'BadValue'}


### Exercise 2.3
##### After using the ``$unwind`` operator on the products contained in order 473748, try to restore the original data.

In [23]:
match = {"$match": {"order_id": 473748}}
unwind = {"$unwind": "$products"}
group = {"$group": {"_id": "$order_id", "products": { "$push": {"product_id": "$products.product_id",
                                                                "product_name": "$products.product_name",
                                                                "aisle_id": "$products.aisle_id",
                                                                "aisle": "$products.aisle",
                                                                "department_id": "$products.department_id",
                                                                "department": "$products.department",
                                                                "add_to_cart_order": "$products.add_to_cart_order",
                                                                "reordered": "$products.reordered"}}}}

query(cll, [match, unwind, group]).to_dict()

{'_id': {0: 473748},
 'products': {0: [{'product_id': 6188,
    'product_name': 'Organic Apple & Butternut Squash Baby Food',
    'aisle_id': 92,
    'aisle': 'baby food formula',
    'department_id': 18,
    'department': 'babies',
    'add_to_cart_order': 10},
   {'product_id': 6347,
    'product_name': 'Unsweetened Almond Milk',
    'aisle_id': 91,
    'aisle': 'soy lactosefree',
    'department_id': 16,
    'department': 'dairy eggs',
    'add_to_cart_order': 14},
   {'product_id': 7051,
    'product_name': 'Happy Tot Banana, Peach, Prune & Coconut Organic Superfoods',
    'aisle_id': 92,
    'aisle': 'baby food formula',
    'department_id': 18,
    'department': 'babies',
    'add_to_cart_order': 2},
   {'product_id': 11712,
    'product_name': 'Cage Free Large White Eggs',
    'aisle_id': 86,
    'aisle': 'eggs',
    'department_id': 16,
    'department': 'dairy eggs',
    'add_to_cart_order': 13,
    'reordered': 1},
   {'product_id': 13176,
    'product_name': 'Bag of Organic 

### Exercise 2.4
##### Select all order ids for user 1 and put them in a list

In [23]:
match = {"$match": {"user_id": 1}}
group = {"$group": {"_id": "$user_id", "products": { "$push": "$order_id"}}}
project = {"$project": {"_id": 0}}

res = query(cll, [match, group, project])
res

,products
0,"[2539329, 473747, 431534, 2398795, 2550362, 33..."


In [24]:
res.iloc[0,0]

[2539329,
 473747,
 431534,
 2398795,
 2550362,
 3367565,
 2254736,
 550135,
 1187899,
 3108588,
 2295261]

### Exercise 2.5
##### String search with PyMongo: How many orders can produce _some sort_ of Caprese, i.e. how many orders contain both Tomatoes and Mozzarella?

The ``$text`` operator works only at the beginning of a pipeline, better use an alternative from Python library ``re``.
``re`` (with its variant ``regex``) is a Python library to perform operations with Regular Expressions, i.e. string patterns. On the web you can find several interfaces to practice with and test your regexs (e.g. https://regexr.com/).

In [24]:
import re
import regex

In [25]:
# Using $unwind to retrieve products that contain the word "tomato"
unwind = {"$unwind": "$products"}
group = {"$group": {"_id": "$products.product_name"}}
match = {"$match": {"_id": re.compile(".*tomato.*", re.IGNORECASE)}}

query(cll, [unwind, group, match])

,_id
0,Tomato Basil Cream Sauce
1,Cocktail Tomatoes
2,100% Natural Tomato Puree
3,Vine Ripe Tomatoes
4,Classic Macaroni & Beef in tomato sauce. Class...
...,...
601,Slow Kettle Tomato & Sweet Basil Bisque Soup
602,100% Natural Tomato Sauce
603,Green House Grown Miraculously Sweet Tomatoes
604,Tuscan Tomato And Basil Bisque


In [26]:
# Using $elemMatch to retrieve the orders that contain products whose name contains the word "mozzarella" and products that contain the word "tomato"
match = {
    "$match": {"$and":
                        [{"products":{"$elemMatch": {"product_name": re.compile(".*mozzarella.*", re.IGNORECASE)}}},
                         {"products":{"$elemMatch": {"product_name": re.compile(".*tomato.*", re.IGNORECASE)}}}]
    }
}

res = query(cll, [match])
res.head()

,_id,order_id,user_id,order_number,order_dow,order_hour_of_day,days_since_prior_order,products
0,60af57782fbbe971834ae469,638062,31,3,4.0,12.0,6.0,"[{'product_id': 6631, 'product_name': 'No Pulp..."
1,60af57782fbbe971834ae473,3035956,31,13,3.0,11.0,6.0,"[{'product_id': 6287, 'product_name': 'Organic..."
2,60af57782fbbe971834ae477,2231262,31,17,3.0,11.0,8.0,"[{'product_id': 196, 'product_name': 'Soda', '..."
3,60af57782fbbe971834ae47c,134879,32,1,3.0,12.0,NaN,"[{'product_id': 2452, 'product_name': 'Natural..."
4,60af57782fbbe971834ae4df,2479273,38,12,1.0,13.0,30.0,"[{'product_id': 300, 'product_name': 'Organic ..."


In [27]:
# Check
[d['product_name'] for d in res.loc[0,"products"]]

['No Pulp Calcium & Vitamin D Pure Orange Juice',
 'Creamy Queso Fresco Chipotle',
 'French Onion Dip',
 'Light Creamy Swiss Cheese',
 'Roasted Garlic Hummus',
 'Cheeseburger Sliders',
 'Creamy Swiss Original Wedges Cheese',
 'Creamy Swiss French Onion Wedges Cheese',
 'Creamy Swiss Garlic & Herb Spreadable Cheese Wedges',
 'Creamy Mozzarella Sun-Dried Tomato & Basil Flavor Cheese Wedges']

In [28]:
# How many
res.shape[0]

18408

### Exercise 2.6
##### On average, how many products does an order include?

In [29]:
unwind = {"$unwind":"$products"}
group1 = {"$group":{"_id":"$order_id", "count":{"$sum":1}}}
group2 = {"$group":{"_id":None, "avg":{"$avg":"$count"}}}

query(cll, [unwind, group1, group2])

,_id,avg
0,None,10.084983


In [30]:
# Solution 1
match = {"$match": {"products": {"$type": "array"}}}
project = {"$project": {"n_products": {"$size": "$products"}}}
group = {"$group": {"_id": None, "avg_products": {"$avg": "$n_products"}}}

query(cll, [match, project, group])

,_id,avg_products
0,None,10.084983


In [30]:
# Solution 2
unwind = {'$unwind': '$products'}
group = {"$group" : {"_id" : "$order_id", "tot_products" : { "$max": "$products.add_to_cart_order"}}}
group1 = {"$group" : {"_id" : None, "avg_products": { "$avg": '$tot_products'}}}

query(cll, [unwind, group, group1])

,_id,avg_products
0,None,10.084983


In [31]:
# Solution 3
unwind = {'$unwind': '$products'}
group = {"$group" : {"_id" : "$order_id", "tot_products" : { "$sum": 1}}}
group1 = {"$group" : {"_id" : None, "avg_products": { "$avg": '$tot_products'}}}

query(cll, [unwind, group, group1])

,_id,avg_products
0,None,10.084983


### Exercise 2.7

##### "Spirits and diapers". 
##### According to a traditional example in microeconomics, spirits and products for babies are complement products (positively correlated). 
##### Get orders that contain at least one item of aisle "spirits" and one of aisle "diapers wipes" or "baby accessories" in the first 5 purchased items

In [32]:
match = {
    "$match": {
        "$and": [
            {"products":{"$elemMatch": {"aisle":"spirits", "add_to_cart_order": {"$lte": 5}}}},
            {"$or": [
                {"products": {"$elemMatch": {"aisle":"diapers wipes", "add_to_cart_order": {"$lte": 5}}}},
                {"products": {"$elemMatch": {"aisle":"baby accessories", "add_to_cart_order": {"$lte": 5}}}}
            ]}
        ]
    }
}
project = {"$project":{"_id":"$order_id", "all_aisles":"$products.aisle"}}
sort = {"$sort":{"_id":1}}

query(cll, [match, project, sort])

,_id,all_aisles
0,137301,"[diapers wipes, frozen pizza, energy sports dr..."
1,425635,"[diapers wipes, spirits]"
2,449709,"[diapers wipes, condiments, condiments, water ..."
3,588085,"[diapers wipes, packaged vegetables fruits, ba..."
4,1235693,"[yogurt, yogurt, frozen breakfast, ice cream i..."
5,1477849,"[packaged vegetables fruits, diapers wipes, cr..."
6,1600786,"[fresh fruits, diapers wipes, specialty wines ..."
7,1851564,"[diapers wipes, crackers, spirits, laundry, pa..."
8,2132910,"[diapers wipes, crackers, paper goods, cleanin..."
9,2463473,"[spirits, soft drinks, diapers wipes, diapers ..."


In [33]:
unwind = {"$unwind": "$products"}
match = {"$match": {"products.add_to_cart_order": {"$lte": 5}}}
group = {"$group": {"_id": "$order_id", "first_five_aisles": {"$push": "$products.aisle"}}}
match1 = {
    "$match": {
        "$or": [{"first_five_aisles": "diapers wipes"}, {"first_five_aisles": "baby accessories"}], 
        "first_five_aisles": "spirits"
    }
}
sort = {"$sort":{"_id":1}}

query(cll, [unwind, match, group, match1, sort])

,_id,first_five_aisles
0,137301,"[diapers wipes, frozen pizza, energy sports dr..."
1,425635,"[diapers wipes, spirits]"
2,449709,"[diapers wipes, condiments, condiments, water ..."
3,588085,"[diapers wipes, baking ingredients, diapers wi..."
4,1235693,"[yogurt, ice cream ice, baby accessories, spir..."
5,1477849,"[diapers wipes, crackers, canned meat seafood,..."
6,1600786,"[diapers wipes, cream, eggs, spirits, yogurt]"
7,1851564,"[diapers wipes, crackers, spirits, laundry, wa..."
8,2132910,"[diapers wipes, crackers, spirits, soup broth ..."
9,2463473,"[spirits, soft drinks, diapers wipes, diapers ..."


### Exercise 2.8
##### What's the most ordered product in each hour of the day?

In [50]:
unwind = {"$unwind": "$products"}
group = {"$group": {"_id":{"product_name": "$products.product_name",
                          "hour": "$order_hour_of_day"}, "n_products":{"$sum":1}}}
sort = {"$sort": {"n_products": -1}}
group1 = {"$group": {"_id": "$_id.hour", "product_name": {"$first": "$_id.product_name"}, "count": {"$first": "$n_products"}}}
sort1 = {"$sort": {"_id": 1}}

In [51]:
query(cll, [unwind, group, sort, group1, sort1])

,_id,product_name,count
0,NaN,Banana,845
1,1.0,Bag of Organic Bananas,444
2,2.0,Banana,250
3,3.0,Bag of Organic Bananas,173
4,4.0,Banana,197
5,5.0,Banana,355
6,6.0,Banana,1384
7,7.0,Banana,4368
8,8.0,Banana,8274
9,9.0,Banana,11496


In [52]:
# Check this result is in line with the overall number of purchases for each product
unwind = {"$unwind": "$products"}
group = {"$group": {"_id":"$products.product_name", "n_products":{"$sum":1}}}
sort = {"$sort":{"n_products":-1}}

query(cll, [unwind, group, sort])

,_id,n_products
0,Banana,144112
1,Bag of Organic Bananas,115186
2,Organic Strawberries,79551
3,Organic Baby Spinach,74061
4,Organic Hass Avocado,65948
...,...,...
47811,Arouses & Intensifies Personal Lubricant,1
47812,"6\"" Organic Carrot Cake",1
47813,Ultra Colloidal Silver,1
47814,Instant Age Rewind Eraser Dark Circles Treatme...,1


### Exercise 2.9
##### Which aisle do customers visit first? Count the number of times each aisle is visited as first.

In [34]:
# Solution 1: unwind after filtering
filter = {
    "$filter":
      {
         "input": "$products",
         "as": "product",
         "cond": { "$eq": ["$$product.add_to_cart_order", 1] }
      }
}
project = {
    "$project": {
        "items": filter
    }
}

unwind = {"$unwind":"$items"}
group = {
    "$group": {
        "_id": "$items.aisle",
        "count":{"$sum":1}
    }
}
sort = {"$sort":{"count":-1}}

query(cll, [project, unwind, group, sort])

,_id,count
0,fresh fruits,136244
1,fresh vegetables,71971
2,milk,57358
3,water seltzer sparkling water,49022
4,packaged vegetables fruits,43280
...,...,...
129,baby accessories,263
130,baby bath body care,258
131,ice cream toppings,257
132,beauty,216


In [35]:
# Solution 2: unwind before filtering
unwind = {"$unwind":"$products"}
match = {"$match":{"products.add_to_cart_order": 1}}
group = {"$group":{"_id":"$products.aisle", "count":{"$sum":1}}}
sort = {"$sort":{"count":-1}}

query(cll, [unwind, match, group, sort])

,_id,count
0,fresh fruits,136244
1,fresh vegetables,71971
2,milk,57358
3,water seltzer sparkling water,49022
4,packaged vegetables fruits,43280
...,...,...
129,baby accessories,263
130,baby bath body care,258
131,ice cream toppings,257
132,beauty,216


### Exercise 2.10
##### Which aisle do customers visit last? Count the number of times each aisle is visited as last.

In [36]:
unwind = {"$unwind": "$products"}
sort = {"$sort": {"products.add_to_cart_order": -1}}
group = {"$group": {"_id": "$order_id", "aisle": {"$first":"$products.aisle"}}}
group1 = {"$group": {"_id": "$aisle", "count": {"$sum": 1}}}
sort = {"$sort": {"count": -1}}

query(cll, [unwind, sort, group, group1, sort])

,_id,count
0,fresh fruits,77448
1,fresh vegetables,74537
2,milk,45055
3,packaged vegetables fruits,36302
4,yogurt,34964
...,...,...
129,baby accessories,263
130,bulk grains rice dried goods,255
131,eye ear care,224
132,beauty,185


### Exercise 2.11
##### Find the average position in which each department is visited (for the first time in an order).

In [38]:
unwind = {"$unwind":"$products"}
group = {"$group":{"_id":"$products.department", "add_to_cart_order":{"$first":"$products.add_to_cart_order"}}}
sort = {"$sort":{"add_to_cart_order":1}}
         
query(cll, [unwind, group, sort])




,_id,add_to_cart_order
0,alcohol,1
1,beverages,1
2,dairy eggs,2
3,bulk,3
4,canned goods,3
5,snacks,3
6,produce,5
7,household,5
8,pantry,5
9,deli,6


In [39]:
unwind = {"$unwind": "$products"}
group = {"$group": {"_id":["$order_id", "$products.department"], "department": {"$first":"$products.department"}, "first_time": {"$min":"$products.add_to_cart_order"}}}
group1 = {"$group": {"_id": "$department", "avg_first_time":{"$avg":"$first_time"}}}
sort = {"$sort": {"avg_first_time": 1}}

query(cll, [unwind, group, group1, sort])

,_id,avg_first_time
0,produce,3.038901
1,dairy eggs,3.738275
2,alcohol,4.236080
3,beverages,4.733481
4,snacks,5.923149
5,frozen,6.262942
6,pets,6.308059
7,bakery,6.609027
8,household,7.006248
9,deli,7.166300


# Exercises from psycopg notebooks

### Exercise 2.1
##### Retrieve the top 10 departments by number of products ordered.

In [40]:
unwind = {"$unwind": "$products"}
group = {"$group": {"_id": "$products.department", "count": {"$sum":1}}}
sort = {"$sort": {"count": -1}}
limit = {"$limit": 10}

query(cll, [unwind, group, sort, limit])

,_id,count
0,produce,2887994
1,dairy eggs,1638848
2,snacks,877769
3,beverages,819136
4,frozen,676611
5,pantry,570082
6,bakery,358422
7,canned goods,326482
8,deli,321829
9,dry goods pasta,263253


### Exercise 2.2

##### Which are the top 10 most purchased products?

In [41]:
unwind = {"$unwind": "$products"}
group = {"$group": {"_id": "$products.product_name", "count": {"$sum":1}}}
sort = {"$sort": {"count": -1}}
limit = {"$limit": 10}

query(cll, [unwind, group, sort, limit])

,_id,count
0,Banana,144112
1,Bag of Organic Bananas,115186
2,Organic Strawberries,79551
3,Organic Baby Spinach,74061
4,Organic Hass Avocado,65948
5,Organic Avocado,53763
6,Large Lemon,46679
7,Strawberries,43696
8,Limes,42332
9,Organic Raspberries,41827


### Exercise 2.6
##### On average, how many different products are purchased in a single aisle?

In [42]:
unwind = {"$unwind":"$products"}
group = {"$group": {"_id":{"aisle":"$products.aisle", "product":"$products.product_id"}}}
group1 = {"$group": {"_id": "$_id.aisle", "products_per_aisle":{"$sum":1}}}
group2 = {"$group": {"_id": None, "avg_products_per_aisle":{"$avg":"$products_per_aisle"}}}

query(cll, [unwind, group, group1, group2])

,_id,avg_products_per_aisle
0,None,356.835821


In [43]:
group = {"$group": {"_id": "$user_id", "count":{"$sum": 1}}}
group1 = {"$group": {"_id": None, "avg": {"$avg": "count"}}}

### Exercise 2.8
##### Get the Top 10 orders by number of departments involved

In [44]:
unwind = {"$unwind":"$products"}
group = {"$group": {"_id": {"order":"$order_id", "department":"$products.department"}}}
group1 = {"$group":{"_id":"$_id.order", "departments_per_order":{"$sum":1}}}
sort = {"$sort": {"departments_per_order":-1}}
limit = {"$limit": 10}

query(cll, [unwind, group, group1, sort, limit])

,_id,departments_per_order
0,1912307,18
1,2595623,17
2,1992955,17
3,2428109,17
4,690270,17
5,1972127,16
6,1261095,16
7,1626876,16
8,2094761,16
9,3381050,16


### Exercise 2.10
##### How many aisles does each department have?

In [45]:
unwind = {"$unwind":"$products"}
group = {"$group": {"_id":{"department": "$products.department", "aisle": "$products.aisle"}}}
group1 = {"$group": {"_id": "$_id.department", "n_aisles":{"$sum":1}}}

query(cll, [unwind, group, group1])

,_id,n_aisles
0,alcohol,5
1,produce,5
2,dry goods pasta,5
3,deli,5
4,other,1
5,household,10
6,breakfast,4
7,beverages,8
8,pets,2
9,personal care,17


In [46]:
# Qualitative check
group2 = {"$group": {"_id": "$_id.department", "aisles":{"$push":"$_id.aisle"}}}

query(cll, [unwind, group, group2])

,_id,aisles
0,deli,"[lunch meat, prepared soups salads, prepared m..."
1,produce,"[fresh vegetables, fresh fruits, packaged prod..."
2,dry goods pasta,"[dry pasta, instant foods, grains rice dried g..."
3,other,[other]
4,beverages,"[cocoa drink mixes, tea, water seltzer sparkli..."
5,household,"[air fresheners candles, more household, plate..."
6,breakfast,"[cereal, hot cereal pancake mixes, granola, br..."
7,personal care,"[skin care, body lotions soap, beauty, shave n..."
8,pets,"[dog food care, cat food care]"
9,frozen,"[frozen meat seafood, frozen breakfast, frozen..."


In [47]:
# Quantitative check
df1 = query(cll, [unwind, group, group1]).set_index("_id")
df2 = query(cll, [unwind, group, group2]).set_index("_id")

df2['n_aisles'] = df2['aisles'].apply(len)
pd.merge(df1, df2, left_index = True, right_index = True)

,n_aisles_x,aisles,n_aisles_y
_id,,,
alcohol,5,"[beers coolers, spirits, red wines, white wine...",5
other,1,[other],1
deli,5,"[tofu meat alternatives, fresh dips tapenades,...",5
dry goods pasta,5,"[pasta sauce, fresh pasta, grains rice dried g...",5
produce,5,"[packaged vegetables fruits, fresh herbs, pack...",5
household,10,"[paper goods, trash bags liners, kitchen suppl...",10
breakfast,4,"[breakfast bars pastries, granola, hot cereal ...",4
beverages,8,"[juice nectars, coffee, soft drinks, energy sp...",8
pets,2,"[cat food care, dog food care]",2


### Exercise 2.11
##### Are there any cart order patterns?
1. <b>Find the name of the top-20 most ordered products as the first item in a single order</b></br>
2. <b>Find the name of the top-20 most ordered products as the third item in a single order</b></br>

In [48]:
# 1.
unwind = {"$unwind":"$products"}
match_first = {"$match": {"products.add_to_cart_order": 1}}
group = {"$group": {"_id":"$products.product_name", "n_purchases":{"$sum": 1}}}
sort = {"$sort": {"n_purchases":-1}}
limit = {"$limit": 20}

query(cll, [unwind, match_first, group, sort, limit])

,_id,n_purchases
0,Banana,34291
1,Bag of Organic Bananas,24087
2,Organic Whole Milk,9428
3,Organic Strawberries,8285
4,Organic Hass Avocado,7496
5,Organic Baby Spinach,7091
6,Organic Avocado,6927
7,Strawberries,5138
8,Spring Water,5001
9,Organic Raspberries,4605


In [49]:
# 2.
unwind = {"$unwind":"$products"}
match_third = {"$match": {"products.add_to_cart_order": 3}}
group = {"$group": {"_id":"$products.product_name", "n_purchases":{"$sum": 1}}}
sort = {"$sort": {"n_purchases":-1}}
limit = {"$limit": 20}

query(cll, [unwind, match_third, group, sort, limit])

,_id,n_purchases
0,Banana,17884
1,Bag of Organic Bananas,15054
2,Organic Strawberries,8220
3,Organic Hass Avocado,7509
4,Organic Baby Spinach,7423
5,Organic Avocado,6191
6,Strawberries,4783
7,Organic Whole Milk,4637
8,Organic Raspberries,4494
9,Large Lemon,4358
